In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torchvision import datasets,transforms
import torch.nn as nn
import torch.optim as optim
from PIL import Image, UnidentifiedImageError
import torchvision.models as models

In [19]:
torch.manual_seed(42)

In [21]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# our model name is resnet
resnet = models.resnet18(pretrained=True)

c:\Users\pc\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pc\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# transforms

In [ ]:
train_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
test_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
# !unzip archive\ \(3\).zip

In [ ]:
train_dataset=datasets.ImageFolder(
    root="/content/Fruits Classification/train",
    transform=train_transform
)
test_dataset=datasets.ImageFolder(
    root="/content/Fruits Classification/test",
    transform=test_transform
)

In [ ]:
train_loader=DataLoader(train_dataset,
                        batch_size=32,
                        shuffle=True)
test_loader=DataLoader(test_dataset,
                        batch_size=32,
                        shuffle=True)

In [ ]:
for p in resnet.parameters():
  p.requires_grad=False

In [ ]:
resnet.fc=nn.Linear(512,5)

In [ ]:
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# training pipeline

In [ ]:
ep=10
lr=0.001
opt=optim.Adam(resnet.fc.parameters(),lr=lr)
cri=nn.CrossEntropyLoss()
for e in range(ep):
    resnet.train()
    loss_ep=0.0
    for im,lb in train_loader:
        im,lb=im.to(device),lb.to(device)
        pred=resnet(im)
        loss=cri(pred,lb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        loss_ep+=loss.item()
    avg_loss=loss_ep/len(train_loader)
    print(f" epoch : {e} |  Loss : {avg_loss}")

 epoch : 0 |  Loss : 0.8117418439176521
 epoch : 1 |  Loss : 0.5567787389614081
 epoch : 2 |  Loss : 0.5215397116875178
 epoch : 3 |  Loss : 0.4885109136077134
 epoch : 4 |  Loss : 0.47027795016765594
 epoch : 5 |  Loss : 0.4747939552425554
 epoch : 6 |  Loss : 0.46441735219406455
 epoch : 7 |  Loss : 0.4599066425703074
 epoch : 8 |  Loss : 0.44632998103962135
 epoch : 9 |  Loss : 0.4463758746554193


In [ ]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
total=00.00
correct=0.0
with torch.no_grad():
  for im,lb in train_loader:
      im,lb=im.to(device),lb.to(device)
      pre=resnet(im)
      _,out=torch.max(pre,1)
      total+=lb.size(0)
      correct+=(out==lb).sum().item()
accuracy = 100 * correct / total
print(f"Train Accuracy: {accuracy:.2f}%")

Train Accuracy: 85.46%


In [ ]:
total=00.00
correct=0.0
with torch.no_grad():
  for im,lb in test_loader:
      im,lb=im.to(device),lb.to(device)
      pre=resnet(im)
      _,out=torch.max(pre,1)
      total+=lb.size(0)
      correct+=(out==lb).sum().item()
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 86.75%


# save model 

In [ ]:
# torch.save(resnet.state_dict(), "fruit_classifier_resnet18.pth")
# print("Model saved as fruit_classifier_resnet18.pth")

Model saved as fruit_classifier_resnet18.pth
